In [1]:
import fractions

import numpy as np

In [2]:
print(f"numpy.__version__: {np.__version__}")

numpy.__version__: 1.18.0


When using the [VS algorithm][1], it's necessary to compute $\binom{n}{k}$. When using Python, one can use an `int`, which has effectively infinite precision, but when using Fortran or C++ for speed, these values can overflow.

In order to avoid storing past values, a single value is tracked and updated via
$$\binom{n}{k + 1} = \frac{n - k}{k + 1} \binom{n}{k}.$$
When using this algorithm, the "numerators" $(n - k) \binom{n}{k}$ may begin to overflow before the values $\binom{n}{k + 1}$ themselves do. (Though once an $n$ is reached where a numerator overflows, the values will overflow at $n + \Delta$ for a very small $\Delta$.)

[1]: https://doi.org/10.1016/0167-8396(86)90018-X

In [3]:
def pascals_triangle(n, dtype):
    is_python_int = dtype is int

    shape = (n + 1,)
    zeros_dtype = object if is_python_int else dtype
    triangle_row = np.zeros(shape, dtype=zeros_dtype)
    n_value = dtype(n)

    triangle_row[0] = dtype(1)
    for k in range(n):
        k_value = dtype(k)
        numerator = triangle_row[k] * (n_value - k_value)
        denominator = k_value + 1
        if is_python_int:
            quotient, remainder = divmod(numerator, denominator)
            if remainder != 0:
                raise ValueError("Unexpected rounding", n, k)
            triangle_row[k + 1] = quotient
        else:
            triangle_row[k + 1] = numerator / denominator

    return triangle_row

## Fortran `c_int` / `c_int32_t`

A 32-bit **signed** integer has maximum value $2^{31} - 1$. The first overflow occurs when computing $\binom{30}{15}$:
$$16 \binom{30}{14} = 2^{31} + 179279152.$$
The first entry in Pascal's triangle to overflow the type is
$$\binom{34}{16} = 2^{31} + 56477782.$$

In [4]:
print(np.iinfo(np.intc), end="")
print(f"2^{{31}} - 1 = {2**31 - 1}")

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------
2^{31} - 1 = 2147483647


In [5]:
print(pascals_triangle(1, dtype=np.intc))
print(pascals_triangle(3, dtype=np.intc))
print(pascals_triangle(29, dtype=np.intc))

[1 1]
[1 3 3 1]
[       1       29      406     3654    23751   118755   475020  1560780
  4292145 10015005 20030010 34597290 51895935 67863915 77558760 77558760
 67863915 51895935 34597290 20030010 10015005  4292145  1560780   475020
   118755    23751     3654      406       29        1]


In [6]:
print(pascals_triangle(30, dtype=np.intc))

[         1         30        435       4060      27405     142506
     593775    2035800    5852925   14307150   30045015   54627300
   86493225  119759850  145422675 -131213633 -123012780 -101304642
  -73164463  -46209134  -25415023  -12102391   -4950978   -1722079
    -502273    -120545     -23181      -3434       -367        -25
          0]


${HERE}/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in int_scalars
  if sys.path[0] == '':


## Fortran `c_int64_t`

A 64-bit **signed** integer has maximum value $2^{63} - 1$. The first overflow occurs when computing $\binom{62}{28}$:
$$35 \binom{62}{27} = 2^{63} + 565868026766073212.$$
The first entry in Pascal's triangle to overflow the type is
$$\binom{67}{30} = 2^{63} + 766318715327501328.$$

In [7]:
print(np.iinfo(np.int64), end="")
print(f"2^{{63}} - 1 = {2**63 - 1}")

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------
2^{63} - 1 = 9223372036854775807


In [8]:
print(pascals_triangle(1, dtype=np.int64))
print(pascals_triangle(3, dtype=np.int64))
print(pascals_triangle(61, dtype=np.int64))

[1 1]
[1 3 3 1]
[                 1                 61               1830
              35990             521855            5949147
           55525372          436270780         2944827765
        17341763505        90177170226       418094152866
      1742058970275      6566222272575     22512762077400
     70539987842520    202802465047245    536830054536824
   1312251244423347   2969831763694943   6236646703759380
  12176310231149266  22138745874816848  37539612570341608
  59437719903040872  87967825456500496 121801604478231456
 157890968768077792 191724747789808736 218169540588403040
 232714176627629920 232714176627629920 218169540588403040
 191724747789808736 157890968768077792 121801604478231440
  87967825456500480  59437719903040864  37539612570341600
  22138745874816840  12176310231149262   6236646703759378
   2969831763694941   1312251244423346    536830054536823
    202802465047244     70539987842519     22512762077399
      6566222272574      1742058970274       41809415286

In [9]:
print(pascals_triangle(62, dtype=np.int64))

[                  1                  62                1891
               37820              557845             6471002
            61474519           491796152          3381098545
         20286591270        107518933731        508271323092
       2160153123141       8308281242850      29078984349975
      93052749919920     273342452889765     739632519584070
    1849081298960175    4282083008118300    9206478467454346
   18412956934908692   34315056105966196   59678358445158600
   96977332473382720  147405545359541728  209769429934732448
  279692573246309952 -309196571788882240  273588297685777760
  300947127454355520 -284401161134521760 -275513624849067968
 -250466931680970880 -213633559374945760 -170906847499956608
 -128180135624967456  -90072527736463616  -59258241931883960
  -36466610419620896  -20968300991282016  -11251283458736692
   -5625641729368346   -2616577548543416   -1129885759598293
    -451954303839317    -167026590549312     -56860115931680
     -17768786228650    

${HERE}/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in long_scalars
  if sys.path[0] == '':


## Fortran `c_double`

A 64-bit (double precision) floating-point number can represent all integers until $2^{53}$, at which point rounding must occur to fit into the 52-bit mantissa. The first time $2^{53}$ is exceeded occurs when computing $\binom{52}{24}$:
$$29 \binom{52}{23} = \mathtt{1.22d89e0b2f2b00}_{16} \cdot 2^{53}.$$
however first roundoff does not occur until $\binom{55}{21}$:
$$35 \binom{55}{20} = \mathtt{1.f6640c0aa061b8}_{16} \cdot 2^{53} \neq \mathtt{1.f6640c0aa061c0}_{16} \cdot 2^{53}.$$
The first entry in Pascal's triangle that requires roundoff is
$$\binom{57}{25} = \mathtt{1.1a366b62211ad8}_{16} \cdot 2^{53} \neq \mathtt{1.1a366b62211ae0}_{16} \cdot 2^{53}.$$

In [10]:
print(np.finfo(np.double), end="")
print(f"2.0^{{52}} - 1 = {2.0**52 - 1}")
print(f"    2.0^{{52}} = {2.0**52}")
print(f"2.0^{{52}} + 1 = {2.0**52 + 1}")
print(f"2.0^{{52}} + 2 = {2.0**52 + 2}")
print(f"2.0^{{53}} - 1 = {2.0**53 - 1}")
print(f"    2.0^{{53}} = {2.0**53}")
print(f"2.0^{{53}} + 1 = {2.0**53 + 1}")
print(f"2.0^{{53}} + 2 = {2.0**53 + 2}")
print('-' * 63)
print(f"np.double(9007199254740993) = {np.double(9007199254740993)}")

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
---------------------------------------------------------------
2.0^{52} - 1 = 4503599627370495.0
    2.0^{52} = 4503599627370496.0
2.0^{52} + 1 = 4503599627370497.0
2.0^{52} + 2 = 4503599627370498.0
2.0^{53} - 1 = 9007199254740991.0
    2.0^{53} = 9007199254740992.0
2.0^{53} + 1 = 9007199254740992.0
2.0^{53} + 2 = 9007199254740994.0
---------------------------------------------------------------
np.double(9007199254740993) = 9007199254740992.0


In [11]:
def maybe_fraction(float_value):
    result = fractions.Fraction(float(float_value))
    if result.denominator == 1:
        return result.numerator

    return result


def pascals_double(n):
    values = pascals_triangle(n, dtype=np.double)
    return np.array([maybe_fraction(value) for value in values])

In [12]:
print(pascals_double(1))
print(pascals_double(3))

[1 1]
[1 3 3 1]


In [13]:
for n in range(51, 54 + 1):
    same = np.all(pascals_double(n) == pascals_triangle(n, dtype=int))
    print(f"{n}: {same}")

51: True
52: True
53: True
54: True


In [14]:
pascals_double(55) - pascals_triangle(55, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, Fraction(-1, 2), Fraction(-1, 2), Fraction(-1, 2),
       Fraction(-1, 2), Fraction(-1, 2), Fraction(-1, 2), Fraction(-1, 2),
       Fraction(-1, 4), Fraction(-1, 8), Fraction(-1, 16),
       Fraction(-1, 16), Fraction(-1, 32), Fraction(-1, 64),
       Fraction(-1, 128), Fraction(-1, 256), Fraction(-3, 2048),
       Fraction(-1, 2048), Fraction(-3, 16384), Fraction(-3, 65536),
       Fraction(-3, 262144), Fraction(-3, 1048576), Fraction(-1, 2097152),
       Fraction(-3, 33554432), Fraction(-1, 67108864),
       Fraction(-1, 536870912), Fraction(-3, 17179869184),
       Fraction(-1, 68719476736), Fraction(-1, 1099511627776),
       Fraction(-5, 140737488355328), Fraction(-3, 4503599627370496)],
      dtype=object)

## Fortran `c_float`

A 32-bit (single precision) floating-point number can represent all integers until $2^{24}$, at which point rounding must occur to fit into the 23-bit mantissa. The first time $2^{24}$ is exceeded occurs when computing $\binom{24}{10}$:
$$15 \binom{24}{9} = \mathtt{1.2b4390}_{16} \cdot 2^{24}.$$
however first roundoff does not occur until $\binom{25}{9}$:
$$17 \binom{25}{8} = \mathtt{1.188f57}_{16} \cdot 2^{24} \neq \mathtt{1.188f58}_{16} \cdot 2^{24}.$$
The first entry in Pascal's triangle that requires roundoff is
$$\binom{28}{12} = \mathtt{1.d032fb}_{16} \cdot 2^{24} \neq \mathtt{1.d032fc}_{16} \cdot 2^{24}.$$

In [15]:
print(np.finfo(np.float32), end="")
one = np.float32(1.0)
two = np.float32(2.0)
value = np.float32(2 ** 23)
print(f"2.0^{{23}} - 1 = {value - one}")
print(f"    2.0^{{23}} = {value}")
print(f"2.0^{{23}} + 1 = {value + one}")
print(f"2.0^{{23}} + 2 = {value + two}")
value = np.float32(2 ** 24)
print(f"2.0^{{24}} - 1 = {value - one}")
print(f"    2.0^{{24}} = {value}")
print(f"2.0^{{24}} + 1 = {value + one}")
print(f"2.0^{{24}} + 2 = {value + two}")
print("-" * 63)
print(f"np.float32(16777217) = {np.float32(16777217)}")

Machine parameters for float32
---------------------------------------------------------------
precision =   6   resolution = 1.0000000e-06
machep =    -23   eps =        1.1920929e-07
negep =     -24   epsneg =     5.9604645e-08
minexp =   -126   tiny =       1.1754944e-38
maxexp =    128   max =        3.4028235e+38
nexp =        8   min =        -max
---------------------------------------------------------------
2.0^{23} - 1 = 8388607.0
    2.0^{23} = 8388608.0
2.0^{23} + 1 = 8388609.0
2.0^{23} + 2 = 8388610.0
2.0^{24} - 1 = 16777215.0
    2.0^{24} = 16777216.0
2.0^{24} + 1 = 16777216.0
2.0^{24} + 2 = 16777218.0
---------------------------------------------------------------
np.float32(16777217) = 16777216.0


In [16]:
def pascals_float(n):
    values = pascals_triangle(n, dtype=np.float32)
    return np.array([maybe_fraction(value) for value in values])

In [17]:
print(pascals_float(1))
print(pascals_float(3))

[1 1]
[1 3 3 1]


In [18]:
for n in range(22, 24 + 1):
    same = np.all(pascals_float(n) == pascals_triangle(n, dtype=int))
    print(f"{n}: {same}")

22: True
23: True
24: True


In [19]:
pascals_float(25) - pascals_triangle(25, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, Fraction(1, 8), Fraction(1, 4),
       Fraction(1, 2), Fraction(1, 2), Fraction(1, 2), Fraction(1, 2),
       Fraction(1, 4), Fraction(1, 8), 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=object)